In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
import sys
sys.executable

'/n/groups/datta/tim_sainburg/conda_envs/mmdeploy/bin/python3'

### important
- must be `module load cuda/11.7`

##### Duration: ???

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from pathlib import Path
import sys
import logging
logging.basicConfig(level=logging.INFO)
print(sys.executable)

/n/groups/datta/tim_sainburg/conda_envs/mmdeploy/bin/python3


In [5]:
from multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D import Inferencer2D

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Python interpreter binary location: /n/groups/datta/tim_sainburg/conda_envs/mmdeploy/bin/python3


### Get recording info (google sheets)

In [6]:
import requests
import pandas as pd
from io import BytesIO

In [7]:
# spreadsheet_url = 'https://docs.google.com/spreadsheet/ccc?key=14HIqUaSl_n-91hpAvmACY_iVY9nLKdlA6qklhxfZon0&output=csv&gid=0'
spreadsheet_url = "https://docs.google.com/spreadsheet/ccc?key=1jACsUmxuJ9Une59qmvzZGc1qXezKhKzD1zho2sEfcrU&output=csv&gid=0"
response = requests.get(spreadsheet_url)
recording_df = pd.read_csv(BytesIO(response.content))

In [8]:
recording_df[:3]

,Subject,duration_m,video_recording_id,ephys_id,calibration_id,video_location_on_o2,ephys_location_on_o2,calibration_location_on_o2,samplerate,username,n_ephys_streams,max_video_duration_m
0,M04002,10,24-05-01-13-26-43-110846,2024-05-01_13-26-37,24-05-01-13-45-07-825493,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,150,tis697,1,10


### Run 2D prediction

In [9]:
for idx, recording_row in recording_df.iterrows():
    break

In [10]:
output_directory = Path("/n/groups/datta/tim_sainburg/datasets/scratch/") / "240806-3d-pipeline"

In [11]:
output_directory_2D = output_directory/ "2D_predictions" 

In [12]:
output_directory_2D.mkdir(parents=True, exist_ok=True)

In [13]:
recording_row.video_location_on_o2

'/n/groups/datta/tim_sainburg/datasets/chronic24/video/M04002/'

In [14]:
video_directory = Path(recording_row.video_location_on_o2) / recording_row.video_recording_id

In [15]:
pose_checkpoint = "/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmpose/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/best_PCK_epoch_200.pth"
pose_config = '/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmpose/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/config.py'
det_checkpoint = '/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmdet/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/epoch_400.pth'
det_config = '/n/groups/datta/tim_sainburg/projects/24-01-05-multicamera_keypoints_mm2d/models/rtmdet/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/config.py'

In [ ]:
#tensorrt_path = "/n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/tensorrt"
rtmpose_model_name = 'rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524'
rtmdetection_model_name = 'rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661'

In [17]:
output_directory_predictions = output_directory_2D / recording_row.video_recording_id

In [ ]:
tensorrt_model_directory = output_directory / "tensorrt"

In [18]:
output_directory_predictions = output_directory_2D / (recording_row.video_recording_id + '_tensorrt')

In [19]:
output_directory_predictions

PosixPath('/n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/2D_predictions/24-05-01-13-26-43-110846_tensorrt')

In [ ]:
expected_video_length_frames = recording_row.max_video_duration_m * recording_row.samplerate

In [20]:
inferencer = Inferencer2D(
    video_directory,
    output_directory_predictions = output_directory_predictions,
    pose_estimator_config = pose_config,
    pose_estimator_checkpoint = pose_checkpoint,
    detector_config = det_config,
    detector_checkpoint = det_checkpoint,
    tensorrt_rtmdetection_model_name = rtmdetection_model_name,
    tensorrt_rtmpose_model_name = rtmpose_model_name,
    tensorrt_model_directory = tensorrt_model_directory,
    use_tensorrt = True,
)

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Using CUDA device: NVIDIA_L40S
INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Using tensorrt, loading models
loading libmmdeploy_trt_net.so ...
loading libmmdeploy_ort_net.so ...
INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Init completed


In [ ]:
inferencer.run()

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Video processing incomplete, running
INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:tensorrt models loaded


[2024-08-13 09:48:18.714] [mmdeploy] [info] [model.cpp:35] [DirectoryModel] Load model: "/n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/tensorrt/rtmdet_tiny_8xb32-300e_coco_chronic_24-05-04-17-51-58_216661/NVIDIA_L40S"
[2024-08-13 09:48:18.887] [mmdeploy] [info] [model.cpp:35] [DirectoryModel] Load model: "/n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline/tensorrt/rtmpose-m_8xb64-210e_ap10k-256x256_24-05-04-21-35-13_305524/NVIDIA_L40S"


videos:   0%|          | 0/12 [00:00<?, ?it/s]

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Manually counting frames for /n/groups/datta/tim_sainburg/datasets/chronic24/video/M04002/24-05-01-13-26-43-110846/24-05-01-13-26-43-110846.BackBottom.0.mp4


counting frames: 0it [00:00, ?it/s]

...

frames:   0%|          | 0/9 [00:00<?, ?it/s]

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Manually counting frames for /n/groups/datta/tim_sainburg/datasets/chronic24/video/M04002/24-05-01-13-26-43-110846/24-05-01-13-26-43-110846.FrontBottom.0.mp4


counting frames: 0it [00:00, ?it/s]

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Total frames: 90000


frames:   0%|          | 0/90000 [00:00<?, ?it/s]